In [1]:
#!pip install torch
#!pip install linear_attention_transformer

In [2]:
import torch

x = torch.rand(5, 3)
print(x)

tensor([[0.3375, 0.9379, 0.0240],
        [0.4986, 0.6026, 0.4952],
        [0.4858, 0.5134, 0.3819],
        [0.3687, 0.0606, 0.2283],
        [0.8407, 0.3845, 0.0461]])


In [3]:
import argparse
import torch
import datetime
import json
import yaml
import os
import pandas as pd
import pickle

from dataset_pm25 import get_dataloader
from main_model import CSDI_PM25
from utils import train, evaluate

In [4]:
parser = argparse.ArgumentParser(description="CSDI")


In [5]:
parser.add_argument("--config", type=str, default="base.yaml")
parser.add_argument('--device', default='cuda:0', help='Device for Attack')
parser.add_argument("--modelfolder", type=str, default="")
parser.add_argument(
    "--targetstrategy", type=str, default="mix", choices=["mix", "random", "historical"]
)
parser.add_argument(
    "--validationindex", type=int, default=0, help="index of month used for validation (value:[0-7])"
)
parser.add_argument("--nsample", type=int, default=100)
parser.add_argument("--unconditional", action="store_true")

_StoreTrueAction(option_strings=['--unconditional'], dest='unconditional', nargs=0, const=True, default=False, type=None, choices=None, required=False, help=None, metavar=None)

In [ ]:
args = parser.parse_args()
print(args)

In [ ]:
path = "config/" + args.config
with open(path, "r") as f:
    config = yaml.safe_load(f)

config["model"]["is_unconditional"] = args.unconditional
config["model"]["target_strategy"] = args.targetstrategy

print(json.dumps(config, indent=4))

In [14]:
import pickle
path = "./save/physio_fold0_20231214_102050/result_nsample100.pk"

with open(path, 'rb') as f:
    data = pickle.load(f)
len(data)

TypeError: object of type 'float' has no len()

### diff_models.py

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from linear_attention_transformer import LinearAttentionTransformer

In [49]:
encoder_layer = nn.TransformerEncoderLayer(d_model=12, nhead=4)#(d_model=512, nhead=8)
src = torch.rand(2, 12)#(10, 32, 512)
out = encoder_layer(src)
out.shape

torch.Size([2, 12])

In [76]:
output

tensor([[-0.1504,  0.0428, -0.0728, -0.2478],
        [ 0.3320,  0.0067,  0.5985,  1.3924],
        [ 0.3325, -0.1921, -0.2349, -1.3469],
        [ 0.1613,  0.9429, -0.0842, -0.0162],
        [-0.9061,  0.3163, -0.1874,  0.0337],
        [-0.3652, -1.0756,  0.0202,  0.0615],
        [ 0.5361, -0.4711,  0.0927, -0.8282],
        [-0.0484,  0.8927,  0.1962,  0.6073]], grad_fn=<AddmmBackward0>)

In [75]:
input

tensor([[ 1.4522e+00, -5.0238e-01, -9.2040e-02, -3.2107e-01, -6.6403e-01,
          1.2625e+00,  6.2727e-01,  1.2717e-01, -9.1781e-02,  1.1118e+00,
          4.8078e-01,  1.2757e+00,  1.9440e+00, -8.5097e-01, -1.9015e+00,
         -3.2794e-01,  4.9607e-01, -6.5903e-01, -6.4518e-01, -6.8441e-01],
        [ 5.8111e-01,  1.3220e+00,  5.2975e-01,  5.8605e-01,  1.1645e+00,
          1.2540e+00, -1.9555e+00,  1.0212e+00,  9.3883e-01, -8.1269e-01,
         -2.7982e-01, -6.6191e-01,  1.1249e+00,  1.4820e+00, -3.1874e-01,
          4.9248e-01, -5.0919e-01,  1.0447e+00,  1.6128e+00,  7.5718e-01],
        [ 1.0264e+00,  9.8663e-01,  4.5151e-01, -1.5656e+00, -1.0930e+00,
          1.2019e-01,  1.3945e+00, -1.2076e+00,  1.5504e-01, -1.2655e+00,
         -8.0773e-01, -1.4208e-01,  4.0462e-01, -1.7291e+00,  1.7125e-02,
         -2.3290e+00, -6.0739e-01, -2.3328e+00, -8.6013e-01,  1.1981e-01],
        [-4.7601e-01,  3.1856e-02, -9.1094e-01,  9.8097e-01,  5.8880e-01,
         -1.3210e+00, -1.6180e+00, 

In [74]:
m = nn.Linear(20, 4)
input = torch.randn(8, 20)
output = m(input)
print(output.size())

torch.Size([8, 4])


In [67]:
m = nn.Conv1d(4, 33, 3, stride=2)
input = torch.randn(2, 4, 4)
output = m(input)

In [17]:
def _build_embedding(self, num_steps, dim=64):
    steps = torch.arange(num_steps).unsqueeze(1)  # (T,1)
    frequencies = 10.0 ** (torch.arange(dim) / (dim - 1) * 4.0).unsqueeze(0)  # (1,dim)
    table = steps * frequencies  # (T,dim)
    table = torch.cat([torch.sin(table), torch.cos(table)], dim=1)  # (T,dim*2)
    return table


In [29]:
num_steps=10 #100
dim=4 # 64
steps = torch.arange(num_steps).unsqueeze(1)
steps

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])

In [34]:
torch.arange(dim) / (dim - 1)

tensor([0.0000, 0.3333, 0.6667, 1.0000])

In [33]:
(torch.arange(dim) / (dim - 1) * 4.0)

tensor([0.0000, 1.3333, 2.6667, 4.0000])

In [32]:
10.0 ** (torch.arange(dim) / (dim - 1) * 4.0)

tensor([1.0000e+00, 2.1544e+01, 4.6416e+02, 1.0000e+04])

In [30]:
frequencies = 10.0 ** (torch.arange(dim) / (dim - 1) * 4.0).unsqueeze(0) 
frequencies

tensor([[1.0000e+00, 2.1544e+01, 4.6416e+02, 1.0000e+04]])

In [38]:
table = steps * frequencies  # (T,dim)
table

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 2.1544e+01, 4.6416e+02, 1.0000e+04],
        [2.0000e+00, 4.3089e+01, 9.2832e+02, 2.0000e+04],
        [3.0000e+00, 6.4633e+01, 1.3925e+03, 3.0000e+04],
        [4.0000e+00, 8.6177e+01, 1.8566e+03, 4.0000e+04],
        [5.0000e+00, 1.0772e+02, 2.3208e+03, 5.0000e+04],
        [6.0000e+00, 1.2927e+02, 2.7850e+03, 6.0000e+04],
        [7.0000e+00, 1.5081e+02, 3.2491e+03, 7.0000e+04],
        [8.0000e+00, 1.7235e+02, 3.7133e+03, 8.0000e+04],
        [9.0000e+00, 1.9390e+02, 4.1774e+03, 9.0000e+04]])

In [39]:
torch.sin(table)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.4321, -0.7151, -0.3056],
        [ 0.9093, -0.7793, -0.9997,  0.5820],
        [ 0.1411,  0.9736, -0.6827, -0.8027],
        [-0.7568, -0.9767,  0.0454,  0.9465],
        [-0.9589,  0.7880,  0.7460, -0.9998],
        [-0.2794, -0.4447,  0.9977,  0.9575],
        [ 0.6570,  0.0140,  0.6488, -0.8235],
        [ 0.9894,  0.4194, -0.0907,  0.6107],
        [ 0.4121, -0.7705, -0.7755, -0.3395]])

In [36]:
table = torch.cat([torch.sin(table), torch.cos(table)], dim=1)  # (T,dim*2)
table

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  1.0000,  1.0000],
        [ 0.8415,  0.4321, -0.7151, -0.3056,  0.5403, -0.9018,  0.6990, -0.9522],
        [ 0.9093, -0.7793, -0.9997,  0.5820, -0.4161,  0.6266, -0.0227,  0.8132],
        [ 0.1411,  0.9736, -0.6827, -0.8027, -0.9900, -0.2284, -0.7307, -0.5964],
        [-0.7568, -0.9767,  0.0454,  0.9465, -0.6536, -0.2147, -0.9990,  0.3226],
        [-0.9589,  0.7880,  0.7460, -0.9998,  0.2837,  0.6156, -0.6659, -0.0179],
        [-0.2794, -0.4447,  0.9977,  0.9575,  0.9602, -0.8957,  0.0680, -0.2885],
        [ 0.6570,  0.0140,  0.6488, -0.8235,  0.7539,  0.9999,  0.7610,  0.5674],
        [ 0.9894,  0.4194, -0.0907,  0.6107, -0.1455, -0.9078,  0.9959, -0.7919],
        [ 0.4121, -0.7705, -0.7755, -0.3395, -0.9111,  0.6375,  0.6313,  0.9406]])

In [ ]:
print(len(data[0]))
print(len(data[0][0]))
print(len(data[0][0][0]))

print(data[0][0][0])

In [7]:
import pickle
#path = "./data/pm25/pm25_meanstd.pk" Documents/GitHub/CSDI/save/physio_fold0_20231214_102050/result_nsample100.pk
path = "./data/physio_missing0.1_seed1.pk"

with open(path, 'rb') as f:
    data = pickle.load(f)
len(data)

3

In [13]:
print(len(data[0]))
print(len(data[0][0]))
print(len(data[0][0][0]))

print(data[0][0][0])

3997
48
35
[-0.         -0.54655684 -0.         -0.         -0.00918508 -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.9443382  -0.         -0.         -0.         -0.         -0.11761455
 -0.         -0.         -0.         -0.         -0.          0.90539649
 -0.          1.72231948 -0.         -0.         -0.         -0.
 -0.         -0.32883745  0.93636527 -0.         -0.        ]


In [18]:
d = 1
print(len(data[d]))
print(len(data[d][0]))
print(len(data[d][0][0]))

print(data[d][0][0])

3997
48
35
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [19]:
d = 2
print(len(data[d]))
print(len(data[d][0]))
print(len(data[d][0][0]))

print(data[d][0][0])

3997
48
35
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [20]:
d = 3
print(len(data[d]))
print(len(data[d][0]))
print(len(data[d][0][0]))

print(data[d][0][0])

IndexError: list index out of range

In [17]:
#print(data[1])
print(data[0][0][0])
print(data[1][0][0])
print(data[2][0][0])

[-0.         -0.54655684 -0.         -0.         -0.00918508 -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.9443382  -0.         -0.         -0.         -0.         -0.11761455
 -0.         -0.         -0.         -0.         -0.          0.90539649
 -0.          1.72231948 -0.         -0.         -0.         -0.
 -0.         -0.32883745  0.93636527 -0.         -0.        ]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]


In [7]:
df = pd.read_csv(
    "./data/pm25/Code/STMVL/SampleData/pm25_ground.txt",
    index_col="datetime",
    parse_dates=True,
)
df.describe()

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
count,7388.000000,7441.000000,8202.000000,7499.000000,7512.000000,8140.000000,7832.000000,7002.000000,8147.000000,7503.000000,...,7958.000000,7437.000000,7549.000000,8194.000000,7478.000000,7186.000000,7394.000000,7087.000000,6898.000000,7069.000000
mean,80.778154,72.105228,82.024019,83.282304,94.015309,81.450614,83.340398,86.009426,82.965386,90.999067,...,69.929756,66.331720,65.980395,67.609104,59.530088,58.153493,69.753449,103.423451,99.623804,121.347715
std,75.446483,70.488158,72.574603,77.068382,87.615338,74.941528,74.759467,78.284668,76.428544,83.066265,...,66.229909,68.063576,63.577960,71.735021,59.248960,60.466758,62.850682,95.728367,91.799252,110.752999
min,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
25%,24.000000,17.000000,27.000000,26.000000,30.000000,24.000000,26.000000,26.000000,23.000000,28.000000,...,18.000000,15.000000,18.000000,14.000000,16.000000,12.000000,20.000000,36.000000,33.000000,42.000000
50%,57.000000,50.000000,62.000000,60.000000,68.000000,60.000000,64.000000,65.000000,62.000000,69.000000,...,50.000000,45.000000,45.000000,42.000000,37.000000,37.000000,52.000000,76.000000,74.000000,86.000000
75%,116.000000,106.000000,117.000000,120.000000,131.000000,116.000000,116.000000,119.000000,117.000000,124.000000,...,101.000000,94.000000,94.000000,96.000000,85.000000,86.000000,101.000000,138.000000,135.000000,164.000000
max,429.000000,470.000000,500.000000,500.000000,500.000000,482.000000,496.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,416.000000,442.000000,440.000000,392.000000,500.000000,500.000000,500.000000,500.000000


In [8]:
df


,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
datetime,,,,,,,,,,,,,,,,,,,,,
2014-05-01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,87.0,...,101.0,84.0,117.0,NaN,97.0,87.0,74.0,94.0,112.0,109.0
2014-05-01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,94.0,...,100.0,77.0,109.0,78.0,97.0,84.0,84.0,101.0,123.0,114.0
2014-05-01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,109.0,...,103.0,90.0,105.0,77.0,103.0,83.0,100.0,112.0,143.0,126.0
2014-05-01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,111.0,...,110.0,94.0,105.0,90.0,107.0,88.0,103.0,120.0,138.0,130.0
2014-05-01 05:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,114.0,...,105.0,80.0,104.0,83.0,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,66.0,...,130.0,136.0,79.0,69.0,74.0,125.0,94.0,67.0,64.0,79.0
2015-04-30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,68.0,...,133.0,124.0,64.0,67.0,75.0,124.0,128.0,103.0,69.0,77.0
2015-04-30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,73.0,...,117.0,123.0,72.0,70.0,82.0,125.0,139.0,166.0,78.0,94.0


In [9]:
test_month = [3, 6, 9, 12]
for i in test_month:
    df = df[df.index.month != i]
    
df

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
datetime,,,,,,,,,,,,,,,,,,,,,
2014-05-01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,87.0,...,101.0,84.0,117.0,NaN,97.0,87.0,74.0,94.0,112.0,109.0
2014-05-01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,94.0,...,100.0,77.0,109.0,78.0,97.0,84.0,84.0,101.0,123.0,114.0
2014-05-01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,109.0,...,103.0,90.0,105.0,77.0,103.0,83.0,100.0,112.0,143.0,126.0
2014-05-01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,111.0,...,110.0,94.0,105.0,90.0,107.0,88.0,103.0,120.0,138.0,130.0
2014-05-01 05:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,114.0,...,105.0,80.0,104.0,83.0,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,66.0,...,130.0,136.0,79.0,69.0,74.0,125.0,94.0,67.0,64.0,79.0
2015-04-30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,68.0,...,133.0,124.0,64.0,67.0,75.0,124.0,128.0,103.0,69.0,77.0
2015-04-30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,73.0,...,117.0,123.0,72.0,70.0,82.0,125.0,139.0,166.0,78.0,94.0


In [ ]:
    def create_normalizer_pm25():
        df = pd.read_csv(
            "./data/pm25/Code/STMVL/SampleData/pm25_ground.txt",
            index_col="datetime",
            parse_dates=True,
        )
        test_month = [3, 6, 9, 12]
        for i in test_month:
            df = df[df.index.month != i]
        mean = df.describe().loc["mean"].values
        std = df.describe().loc["std"].values
        path = "./data/pm25/pm25_meanstd.pk"
        with open(path, "wb") as f:
            pickle.dump([mean, std], f)
    create_normalizer_pm25()

In [2]:

import pandas as pd
# Set some configurations for displaying tables nicely
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

In [7]:
from dataset_physio import get_dataloader, parse_id

In [8]:
id_ = 132539
missing_ratio = 0.1
observed_values, observed_masks, gt_masks = parse_id(
                        id_, missing_ratio
                    )

In [17]:
print(pd.DataFrame(observed_values))

     0     1      2    3     4    5     6    7    8    9   ...     25   26  \
0   0.0  77.0    0.0  0.0  58.0  0.0   0.0  0.0  0.0  0.0  ...  157.0  0.0   
1   0.0  60.0    0.0  0.0  62.0  0.0   0.0  0.0  0.0  0.0  ...  137.0  0.0   
2   0.0  62.0    0.0  0.0  52.0  0.0   0.0  0.0  0.0  0.0  ...  123.0  0.0   
3   0.0  80.0    0.0  0.0  52.0  0.0   0.0  0.0  0.0  0.0  ...  114.0  0.0   
4   0.0  74.0    0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  ...    0.0  0.0   
5   0.0  73.0    0.0  0.0  45.0  0.0   0.0  0.0  0.0  0.0  ...  110.0  0.0   
6   0.0   0.0    0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  ...    0.0  0.0   
7   0.0  64.0    0.0  0.0  49.0  0.0   0.0  0.0  0.0  0.0  ...  107.0  0.0   
8   0.0  64.0    0.0  0.0  56.0  0.0   0.0  0.0  0.0  0.0  ...  102.0  0.0   
9   0.0  66.0    0.0  0.0  48.0  0.0   0.0  0.0  0.0  0.0  ...  114.0  0.0   
10  0.0  61.0  137.0  0.0  62.0  0.0  11.2  0.0  0.0  0.0  ...  109.0  0.0   
11  0.0  58.0    0.0  0.0  40.0  0.0   0.0  0.0  0.0  0.0  ...  

In [10]:
observed_masks

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]], dtype=float32)

In [11]:
gt_masks

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:

C:\Users\fuad>cd Documents\GitHub\CSDI
cd Documents\GitHub\CSDI

python exe_physio.py --testmissingratio [missing ratio] --nsample [number of samples]
python exe_physio.py --nsample 1000
python exe_physio.py